In [2]:
import tensorflow as tf
import numpy as np
from keras import datasets,models,utils,layers
from keras.applications import VGG16

In [3]:
pretrained_model = VGG16(weights='imagenet',include_top=False,input_shape=(32,32,3))
pretrained_model.trainable=False
pretrained_model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                              

In [7]:
base_model = tf.keras.Sequential([
    pretrained_model,
    layers.GlobalAveragePooling2D(),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='sigmoid')
])
base_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d_2  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                                 
Total params: 14848586 (56.64 MB)
Trainable params: 133898 (523.04 KB)
Non-trainable params: 14714688 (56.13 MB)
_______

In [9]:
last_output = base_model.get_layer('dense_5').output
model = tf.keras.Model(base_model.input,last_output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16_input (InputLayer)    [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d_2  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                             

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

170498071/170498071 [==============================] - 5s 0us/step


In [14]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Model accuracy on CIFAR10 test set: {accuracy * 100:.2f}%')

313/313 [==============================] - 3s 8ms/step - loss: 1.1160 - accuracy: 0.6190
Model accuracy on CIFAR10 test set: 61.90%


In [16]:
model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_test,y_test))

Epoch 1/10
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8274 - accuracy: 0.7094 - val_loss: 1.1129 - val_accuracy: 0.6233
Epoch 2/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.8010 - accuracy: 0.7177 - val_loss: 1.1216 - val_accuracy: 0.6180
Epoch 3/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.7757 - accuracy: 0.7266 - val_loss: 1.1370 - val_accuracy: 0.6157
Epoch 4/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.7539 - accuracy: 0.7334 - val_loss: 1.1521 - val_accuracy: 0.6196
Epoch 5/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.7285 - accuracy: 0.7420 - val_loss: 1.1909 - val_accuracy: 0.6121
Epoch 6/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.7078 - accuracy: 0.7499 - val_loss: 1.1850 - val_accuracy: 0.6170
Epoch 7/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.6850 - accuracy: 0.7594 - val_loss: 1.2252 -

In [19]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Fine-tuned model accuracy on test set: {accuracy * 100:.2f}%')

313/313 [==============================] - 3s 9ms/step - loss: 1.2701 - accuracy: 0.6104
Fine-tuned model accuracy on test set: 61.04%
